![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Tutorial  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/mjx/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for [**MuJoCo XLA (MJX)**](https://github.com/google-deepmind/mujoco/blob/main/mjx), a JAX-based implementation of MuJoCo useful for RL training workloads.

**A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".










# Install MuJoCo, MJX, and Brax

In [ ]:
!pip install mujoco
!pip install mujoco_mjx
!pip install brax


In [ ]:
#@title Check if MuJoCo installation was successful

from google.colab import files

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Tell XLA to use Triton GEMM, this improves steps/sec by ~30% on some GPUs
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags


In [ ]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [ ]:
#@title Import MuJoCo, MJX, and Brax
from datetime import datetime
from etils import epath
import functools
from IPython.display import HTML
from typing import Any, Dict, Sequence, Tuple, Union
import os
from ml_collections import config_dict


import jax
from jax import numpy as jp
import numpy as np
from flax.training import orbax_utils
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from orbax import checkpoint as ocp

import mujoco
from mujoco import mjx

from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.base import State as PipelineState
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model

# Simple ENV with spider



## First Version of Spider

In [ ]:
# Generate a simple spider XML model
spider_xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="floor" type="plane" size="10 10 .1" rgba=".9 .9 .9 1"/>
    <body name="torso" pos="0 0 0.25">
      <geom type="sphere" size="0.1" rgba="1 0 0 1"/>
      <joint type="free"/>
      <body name="leg1" pos="0.1 0 0">
        <joint name="joint1" type="hinge" axis="0 1 0" pos="0 0 0"/>
        <geom type="capsule" fromto=".0 .0 .0 .3 .0 .0" size=".02" rgba="0 1 0 1"/>
        <body name="foot1" pos=".3 0 0">
          <joint name="joint2" type="hinge" axis="0 1 0" pos="0 0 0"/>
          <geom type="capsule" fromto=".0 .0 .0 .3 .0 .0" size=".02" rgba="0 1 0 1"/>
        </body>
      </body>
       <body name="leg2" pos="-0.1 0 0">
        <joint name="joint3" type="hinge" axis="0 1 0" pos="0 0 0"/>
        <geom type="capsule" fromto=".0 .0 .0 -.3 .0 .0" size=".02" rgba="0 1 0 1"/>
        <body name="foot2" pos="-.3 0 0">
          <joint name="joint4" type="hinge" axis="0 1 0" pos="0 0 0"/>
          <geom type="capsule" fromto=".0 .0 .0 -.3 .0 .0" size=".02" rgba="0 1 0 1"/>
        </body>
      </body>
       <body name="leg3" pos="0 0.1 0">
        <joint name="joint5" type="hinge" axis="1 0 0" pos="0 0 0"/>
        <geom type="capsule" fromto=".0 .0 .0 .0 .3 .0" size=".02" rgba="0 1 0 1"/>
         <body name="foot3" pos="0 .3 0">
          <joint name="joint6" type="hinge" axis="1 0 0" pos="0 0 0"/>
          <geom type="capsule" fromto=".0 .0 .0 .0 .3 .0" size=".02" rgba="0 1 0 1"/>
        </body>
      </body>
       <body name="leg4" pos="0 -0.1 0">
        <joint name="joint7" type="hinge" axis="1 0 0" pos="0 0 0"/>
        <geom type="capsule" fromto=".0 .0 .0 .0 -.3 .0" size=".02" rgba="0 1 0 1"/>
        <body name="foot4" pos="0 -.3 0">
          <joint name="joint8" type="hinge" axis="1 0 0" pos="0 0 0"/>
          <geom type="capsule" fromto=".0 .0 .0 .0 -.3 .0" size=".02" rgba="0 1 0 1"/>
        </body>
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor name="act1" joint="joint1"/>
    <motor name="act2" joint="joint2"/>
    <motor name="act3" joint="joint3"/>
    <motor name="act4" joint="joint4"/>
    <motor name="act5" joint="joint5"/>
    <motor name="act6" joint="joint6"/>
    <motor name="act7" joint="joint7"/>
    <motor name="act8" joint="joint8"/>
  </actuator>
</mujoco>
"""

## Second Version of spider

In [ ]:
spider_xml = """
<mujoco model="ant">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option integrator="RK4" timestep="0.01"/>
  <custom>
    <numeric data="0.0 0.0 0.55 1.0 0.0 0.0 0.0 0.0 1.0 0.0 -1.0 0.0 -1.0 0.0 1.0" name="init_qpos"/>
  </custom>
  <default>
    <joint armature="1" damping="1" limited="true"/>
    <geom conaffinity="0" condim="3" density="5.0" friction="1 0.5 0.5" margin="0.01" rgba="0.8 0.6 0.4 1"/>
  </default>
  <asset>
    <texture builtin="gradient" height="100" rgb1="1 1 1" rgb2="0 0 0" type="skybox" width="100"/>
    <texture builtin="flat" height="1278" mark="cross" markrgb="1 1 1" name="texgeom" random="0.01" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" type="cube" width="127"/>
    <texture builtin="checker" height="100" name="texplane" rgb1="0 0 0" rgb2="0.8 0.8 0.8" type="2d" width="100"/>
    <material name="MatPlane" reflectance="0.5" shininess="1" specular="1" texrepeat="60 60" texture="texplane"/>
    <material name="geom" texture="texgeom" texuniform="true"/>
  </asset>
  <worldbody>
    <light cutoff="100" diffuse="1 1 1" dir="-0 0 -1.3" directional="true" exponent="1" pos="0 0 1.3" specular=".1 .1 .1"/>
    <geom conaffinity="1" condim="3" material="MatPlane" name="floor" pos="0 0 0" rgba="0.8 0.9 0.8 1" size="40 40 40" type="plane"/>
    <body name="torso" pos="0 0 0.75">
      <camera name="track" mode="trackcom" pos="0 -3 0.3" xyaxes="1 0 0 0 0 1"/>
      <geom name="torso_geom" pos="0 0 0" size="0.25" type="sphere"/>
      <joint armature="0" damping="0" limited="false" margin="0.01" name="root" pos="0 0 0" type="free"/>
      <body name="front_left_leg" pos="0 0 0">
        <geom fromto="0.0 0.0 0.0 0.2 0.2 0.0" name="aux_1_geom" size="0.08" type="capsule"/>
        <body name="aux_1" pos="0.2 0.2 0">
          <joint axis="0 0 1" name="hip_1" pos="0.0 0.0 0.0" range="-30 30" type="hinge"/>
          <geom fromto="0.0 0.0 0.0 0.2 0.2 0.0" name="left_leg_geom" size="0.08" type="capsule"/>
          <body pos="0.2 0.2 0">
            <joint axis="-1 1 0" name="ankle_1" pos="0.0 0.0 0.0" range="30 70" type="hinge"/>
            <geom fromto="0.0 0.0 0.0 0.4 0.4 0.0" name="left_ankle_geom" size="0.08" type="capsule"/>
          </body>
        </body>
      </body>
      <body name="front_right_leg" pos="0 0 0">
        <geom fromto="0.0 0.0 0.0 -0.2 0.2 0.0" name="aux_2_geom" size="0.08" type="capsule"/>
        <body name="aux_2" pos="-0.2 0.2 0">
          <joint axis="0 0 1" name="hip_2" pos="0.0 0.0 0.0" range="-30 30" type="hinge"/>
          <geom fromto="0.0 0.0 0.0 -0.2 0.2 0.0" name="right_leg_geom" size="0.08" type="capsule"/>
          <body pos="-0.2 0.2 0">
            <joint axis="1 1 0" name="ankle_2" pos="0.0 0.0 0.0" range="-70 -30" type="hinge"/>
            <geom fromto="0.0 0.0 0.0 -0.4 0.4 0.0" name="right_ankle_geom" size="0.08" type="capsule"/>
          </body>
        </body>
      </body>
      <body name="back_leg" pos="0 0 0">
        <geom fromto="0.0 0.0 0.0 -0.2 -0.2 0.0" name="aux_3_geom" size="0.08" type="capsule"/>
        <body name="aux_3" pos="-0.2 -0.2 0">
          <joint axis="0 0 1" name="hip_3" pos="0.0 0.0 0.0" range="-30 30" type="hinge"/>
          <geom fromto="0.0 0.0 0.0 -0.2 -0.2 0.0" name="back_leg_geom" size="0.08" type="capsule"/>
          <body pos="-0.2 -0.2 0">
            <joint axis="-1 1 0" name="ankle_3" pos="0.0 0.0 0.0" range="-70 -30" type="hinge"/>
            <geom fromto="0.0 0.0 0.0 -0.4 -0.4 0.0" name="third_ankle_geom" size="0.08" type="capsule"/>
          </body>
        </body>
      </body>
      <body name="right_back_leg" pos="0 0 0">
        <geom fromto="0.0 0.0 0.0 0.2 -0.2 0.0" name="aux_4_geom" size="0.08" type="capsule"/>
        <body name="aux_4" pos="0.2 -0.2 0">
          <joint axis="0 0 1" name="hip_4" pos="0.0 0.0 0.0" range="-30 30" type="hinge"/>
          <geom fromto="0.0 0.0 0.0 0.2 -0.2 0.0" name="rightback_leg_geom" size="0.08" type="capsule"/>
          <body pos="0.2 -0.2 0">
            <joint axis="1 1 0" name="ankle_4" pos="0.0 0.0 0.0" range="30 70" type="hinge"/>
            <geom fromto="0.0 0.0 0.0 0.4 -0.4 0.0" name="fourth_ankle_geom" size="0.08" type="capsule"/>
          </body>
        </body>
      </body>
    </body>
  </worldbody>
  <actuator>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="hip_4" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="ankle_4" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="hip_1" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="ankle_1" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="hip_2" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="ankle_2" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="hip_3" gear="150"/>
    <motor ctrllimited="true" ctrlrange="-1.0 1.0" joint="ankle_3" gear="150"/>
  </actuator>
</mujoco>
"""

## Spider Env

In [ ]:

# HUMANOID_ROOT_PATH = epath.Path(epath.resource_path('mujoco')) / 'mjx/test_data/humanoid'

class Humanoid(PipelineEnv):

  def __init__(
      self,
      forward_reward_weight=10.0,
      ctrl_cost_weight=0.5,
      healthy_reward=2.5,
      terminate_when_unhealthy=False, # Set to False to prevent early termination
      healthy_z_range=(0.1, 0.3), # Lowered healthy_z_range for a spider
      reset_noise_scale=1e-2,
      exclude_current_positions_from_observation=True,
      episode_length: int = 1000, # <--- Added episode_length with default
      **kwargs,
  ):
#


    mj_model = mujoco.MjModel.from_xml_string(spider_xml)
    mj_data = mujoco.MjData(mj_model) # Create mj_data
    mj_model.opt.solver = mujoco.mjtSolver.mjSOL_CG
    mj_model.opt.iterations = 6
    mj_model.opt.ls_iterations = 6

    sys = mjcf.load_model(mj_model)

    physics_steps_per_control_step = 5
    kwargs['n_frames'] = kwargs.get(
        'n_frames', physics_steps_per_control_step)
    kwargs['backend'] = 'mjx'
    # Store episode_length as an attribute of the Humanoid instance
    self.episode_length = episode_length
    super().__init__(sys, **kwargs)

    self._forward_reward_weight = forward_reward_weight
    self._ctrl_cost_weight = ctrl_cost_weight
    self._healthy_reward = healthy_reward
    self._terminate_when_unhealthy = terminate_when_unhealthy
    self._healthy_z_range = healthy_z_range
    self._reset_noise_scale = reset_noise_scale
    self._exclude_current_positions_from_observation = (
        exclude_current_positions_from_observation
    )
    self._torso_body_idx = mujoco.mj_name2id(
        self.sys.mj_model, mujoco.mjtObj.mjOBJ_BODY.value, 'torso'
    )
    # Store mj_data for access to qpos0 and qvel0
    self._mj_data = mj_data


  def reset(self, rng: jp.ndarray) -> State:
    """Resets the environment to an initial state."""
    rng, rng1, rng2 = jax.random.split(rng, 3)

    low, hi = -self._reset_noise_scale, self._reset_noise_scale
    qpos = jp.asarray(self._mj_data.qpos) + jax.random.uniform(
        rng1, (self.sys.nq,), minval=low, maxval=hi
    )
    qvel = jp.asarray(self._mj_data.qvel) + jax.random.uniform(
        rng2, (self.sys.nv,), minval=low, maxval=hi
    )

    data = self.pipeline_init(qpos, qvel)

    obs = self._get_obs(data, jp.zeros(self.sys.nu))
    reward, done, zero = jp.zeros(3)
    metrics = {
        'forward_reward': zero,
        'reward_linvel': zero,
        'reward_quadctrl': zero,
        'reward_alive': zero,
        'x_position': zero,
        'y_position': zero,
        'distance_from_origin': zero,
        'x_velocity': zero,
        'y_velocity': zero,
    }
    return State(data, obs, reward, done, metrics)

  def step(self, state: State, action: jp.ndarray) -> State:
    """Runs one timestep of the environment's dynamics."""
    data0 = state.pipeline_state
    data = self.pipeline_step(data0, action)

    # Calculate forward velocity using the torso's velocity
    torso_velocity = data.cvel[self._torso_body_idx, 0]
    forward_reward = self._forward_reward_weight * torso_velocity


    min_z, max_z = self._healthy_z_range
    is_healthy = jp.where(data.q[2] < min_z, 0.0, 1.0)
    is_healthy = jp.where(data.q[2] > max_z, 0.0, is_healthy)
    if self._terminate_when_unhealthy:
      healthy_reward = self._healthy_reward
    else:
      healthy_reward = self._healthy_reward * is_healthy

    ctrl_cost = self._ctrl_cost_weight * jp.sum(jp.square(action))

    obs = self._get_obs(data, action)
    reward = forward_reward + healthy_reward - ctrl_cost
    done = 1.0 - is_healthy if self._terminate_when_unhealthy else 0.0
    state.metrics.update(
        forward_reward=forward_reward,
        reward_linvel=forward_reward,
        reward_quadctrl=-ctrl_cost,
        reward_alive=healthy_reward,
        x_position=data.xpos[self._torso_body_idx, 0],
        y_position=data.xpos[self._torso_body_idx, 1],
        distance_from_origin=jp.linalg.norm(data.xpos[self._torso_body_idx, :2]),
        x_velocity=torso_velocity,
        y_velocity=data.cvel[self._torso_body_idx, 1],
    )

    return state.replace(
        pipeline_state=data, obs=obs, reward=reward, done=done
    )

  def _get_obs(
      self, data: mjx.Data, action: jp.ndarray
  ) -> jp.ndarray:
    """Observes humanoid body position, velocities, and angles."""
    # Simplified observation: torso z-position and x-velocity
    # return jp.concatenate([
    #     data.qpos[2:3],  # Torso z-position
    #     data.cvel[self._torso_body_idx, 0:1], # Torso x-velocity
    # ])

    # More detailed observation for a spider: torso z-position, torso x-velocity,
    # and joint positions and velocities
    return jp.concatenate([
        data.qpos[2:3],  # Torso z-position
        data.cvel[self._torso_body_idx, 0:1], # Torso x-velocity
        data.qpos[7:], # Joint positions (excluding free joint)
        data.qvel[6:], # Joint velocities (excluding free joint)
    ])


envs.register_environment('humanoid', Humanoid)

# Add code to get and print the observation shape
env_test = Humanoid()
dummy_data = mujoco.MjData(env_test.sys.mj_model) # Use mj_data
dummy_obs = env_test._get_obs(env_test.pipeline_init(jp.asarray(dummy_data.qpos), jp.asarray(dummy_data.qvel)), jp.zeros(env_test.sys.nu))
print("Observation shape:", dummy_obs.shape)


##(Optional) Show Data class for inspection

In [ ]:
# import jax
# import mujoco
# from brax import envs
# import numpy as np

# # Instantiate the environment
# env_name = 'humanoid'
# env = envs.get_environment(env_name)

# # Define the jit reset function
# jit_reset = jax.jit(env.reset)

# # Reset the environment to get an initial state
# rng = jax.random.PRNGKey(0)
# state = jit_reset(rng)

# # The mjx.Data object is stored in state.pipeline_state
# data = state.pipeline_state

# print("--- Contents of mjx.Data object ---")
# print(f"Type of data: {type(data)}\n")

# print("--- All attributes of mjx.Data object ---\n")
# for attr_name in sorted(dir(data)):
#     if not attr_name.startswith('_'): # Exclude private attributes
#         attr_value = getattr(data, attr_name)
#         attr_type = type(attr_value)
#         if isinstance(attr_value, (np.ndarray, jax.Array)): # Check if it's a JAX or NumPy array
#             print(f"  Attribute: {attr_name}, Type: {attr_type}, Shape: {attr_value.shape}")
#         else:
#             print(f"  Attribute: {attr_name}, Type: {attr_type}")

# print("\n--- Specific attributes shown before ---")
# print("1. Generalized positions (qpos):")
# print(f"  Shape: {data.qpos.shape}")
# print(f"  Value (first 10 elements): {data.qpos[:10]}\n")

# print("2. Generalized velocities (qvel):")
# print(f"  Shape: {data.qvel.shape}")
# print(f"  Value (first 10 elements): {data.qvel[:10]}\n")

# print("3. Cartesian position of bodies (xpos):")
# print(f"  Shape: {data.xpos.shape}")
# print(f"  Value (first 5 bodies):\n{data.xpos[:5]}\n")

# print("4. Cartesian orientation of bodies (xquat):")
# print(f"  Shape: {data.xquat.shape}")
# print(f"  Value (first 5 bodies):\n{data.xquat[:5]}\n")

# print("5. Center of mass velocity (cvel):")
# print(f"  Shape: {data.cvel.shape}")
# print(f"  Value (first 5 bodies):\n{data.cvel[:5]}\n")

# print("6. Body indices (for reference, not part of data object directly but useful):")
# print(f"  Torso body index: {env._torso_body_idx}")

## (Optional) Inspect action

In [ ]:
# import jax
# from brax import envs
# import jax.numpy as jp
# from brax.training.agents.ppo import networks as ppo_networks
# from brax.training.agents.ppo import train as ppo
# from brax.io import model

# # Re-instantiate the environment if not already available in this scope
# env_name = 'humanoid'
# env = envs.get_environment(env_name)

# # Define and train a minimal policy if jit_inference_fn is not defined
# try:
#     # Attempt to use jit_inference_fn if already defined (e.g., from a previous run of training cells)
#     _ = jit_inference_fn
# except NameError:
#     print("jit_inference_fn not found, training a minimal policy...")
#     make_inference_fn, params, _ = ppo.train(
#         environment=env, num_timesteps=5_000, num_evals=1, episode_length=env.episode_length # Minimal training just for this demo
#     )
#     model_path = '/tmp/mjx_brax_policy'
#     model.save_params(model_path, params)
#     params = model.load_params(model_path)
#     inference_fn = make_inference_fn(params)
#     jit_inference_fn = jax.jit(inference_fn)
#     print("Minimal policy trained and jit_inference_fn defined.")

# # Initialize the state
# current_rng = jax.random.PRNGKey(1) # Use a new RNG key
# state = jax.jit(env.reset)(current_rng)

# # Generate an action using the inference function
# act_rng, current_rng = jax.random.split(current_rng)
# ctrl, _ = jit_inference_fn(state.obs, act_rng)

# print("--- Content of the 'action' variable (ctrl from policy) ---")
# print(f"Type of action: {type(ctrl)}")
# print(f"Shape of action: {ctrl.shape}")
# print(f"Value of action: {ctrl}")


## Visualize a Rollout

Let's instantiate the environment and visualize a short rollout.

NOTE: Since episodes terminate early if the torso is below the healthy z-range, the only relevant contacts for this task are between the feet and the plane. We turn off other contacts.

In [ ]:
# instantiate the environment
env_name = 'humanoid'
env = envs.get_environment(env_name)

# define the jit reset/step functions
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)


In [ ]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]

# grab a trajectory
for i in range(100):
  ctrl = jp.zeros(env.sys.nu) # Set control input to zero for standing still
  state = jit_step(state, ctrl)
  rollout.append(state.pipeline_state)

media.show_video(env.render(rollout), fps=1.0 / env.dt)

In [ ]:
print("Observation space:", env.observation_size)

# Train Humanoid Policy


In [ ]:
# Importujemy 'networks' bezpośrednio z modułu agenta PPO
from brax.training.agents.ppo import networks as ppo_networks
# Typ PPONetworks jest teraz w 'brax.training.types'
from brax.training.agents.ppo.networks import PPONetworks
from brax.io import html, model
import flax.linen as nn

def my_custom_network_factory(
    observation_size: int,
    action_size: int,
    preprocess_observations_fn=None,
    hidden_layer_sizes=(512, 256),
    activation=nn.relu
) -> PPONetworks:
    """Tworzy sieć PPO o niestandardowej architekturze."""

    return ppo_networks.make_networks(
        observation_size=observation_size,
        action_size=action_size,
        preprocess_observations_fn=preprocess_observations_fn,
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation
    )
train_fn = functools.partial(
    ppo.train,network_factory=my_custom_network_factory, num_timesteps=20_000_00, num_evals=5, reward_scaling=0.1,
    episode_length=1000, normalize_observations=True, action_repeat=1,
    unroll_length=10, num_minibatches=24, num_updates_per_batch=8,
    discounting=0.97, learning_rate=3e-4, entropy_cost=1e-3, num_envs=3072,
    batch_size=512, seed=0)

# starty, nagrody na wykresie
x_data = []
y_data = [] # eval/episode_reward
ydataerr = [] # eval/episode_reward_std

# New lists for other reward components
y_data_forward = []
ydataerr_forward = []
y_data_quadctrl = []
ydataerr_quadctrl = []
y_data_alive = []
ydataerr_alive = []

times = [datetime.now()]

max_y, min_y = 5000, -2000 # Adjusted max_y and min_y for potentially higher/lower rewards
def progress(num_steps, metrics):
  times.append(datetime.now())
  x_data.append(num_steps)
  y_data.append(metrics['eval/episode_reward'])
  ydataerr.append(metrics['eval/episode_reward_std'])

  # Append data for new metrics, using .get() with default 0.0 to handle initial missing keys
  y_data_forward.append(metrics.get('eval/episode_metrics/forward_reward', 0.0))
  ydataerr_forward.append(metrics.get('eval/episode_metrics/forward_reward_std', 0.0))
  y_data_quadctrl.append(metrics.get('eval/episode_metrics/reward_quadctrl', 0.0))
  ydataerr_quadctrl.append(metrics.get('eval/episode_metrics/reward_quadctrl_std', 0.0))
  y_data_alive.append(metrics.get('eval/episode_metrics/reward_alive', 0.0))
  ydataerr_alive.append(metrics.get('eval/episode_metrics/reward_alive_std', 0.0))

  plt.figure(figsize=(12, 16)) # Create a new figure for multiple subplots

  # Plot 1: Total Episode Reward
  plt.subplot(4, 1, 1) # 4 rows, 1 column, first plot
  plt.xlim([0, train_fn.keywords['num_timesteps'] * 1.25])
  plt.ylim([min_y, max_y])
  plt.xlabel('# environment steps')
  plt.ylabel('Reward per episode')
  plt.title(f'Total Episode Reward: {y_data[-1]:.3f}')
  plt.errorbar(x_data, y_data, yerr=ydataerr, label='Total Reward')
  plt.legend()

  # # Plot 2: Forward Reward
  # plt.subplot(4, 1, 2)
  # plt.xlim([0, train_fn.keywords['num_timesteps'] * 1.25])
  # plt.xlabel('# environment steps')
  # plt.ylabel('Forward Reward')
  # plt.title(f'Forward Reward: {y_data_forward[-1]:.3f}')
  # plt.errorbar(x_data, y_data_forward, yerr=ydataerr_forward, label='Forward Reward', color='green')
  # plt.legend()

  # # Plot 3: Control Cost Reward
  # plt.subplot(4, 1, 3)
  # plt.xlim([0, train_fn.keywords['num_timesteps'] * 1.25])
  # plt.xlabel('# environment steps')
  # plt.ylabel('Control Cost Reward')
  # plt.title(f'Control Cost Reward: {y_data_quadctrl[-1]:.3f}')
  # plt.errorbar(x_data, y_data_quadctrl, yerr=ydataerr_quadctrl, label='Control Cost', color='red')
  # plt.legend()

  # # Plot 4: Alive Reward
  # plt.subplot(4, 1, 4)
  # plt.xlim([0, train_fn.keywords['num_timesteps'] * 1.25])
  # plt.xlabel('# environment steps')
  # plt.ylabel('Alive Reward')
  # plt.title(f'Alive Reward: {y_data_alive[-1]:.3f}')
  # plt.errorbar(x_data, y_data_alive, yerr=ydataerr_alive, label='Alive Reward', color='purple')
  # plt.legend()

  plt.tight_layout() # Adjust layout to prevent overlap
  plt.show()

print("Observation space size:", env.observation_size)
print("Action space size:", env.action_size)
make_inference_fn, params, _= train_fn(environment=env, progress_fn=progress)

print(f'time to jit: {times[1] - times[0]}')
print(f'time to train: {times[-1] - times[1]}')

<!-- ## Save and Load Policy -->

We can save and load the policy using the brax model API.

In [ ]:
#@title Save Model
model_path = '/tmp/mjx_brax_policy'
model.save_params(model_path, params)

In [ ]:
#@title Load Model and Define Inference Function
params = model.load_params(model_path)

inference_fn = make_inference_fn(params)
jit_inference_fn = jax.jit(inference_fn)

## Visualize Policy

Finally we can visualize the policy.

In [ ]:
eval_env = envs.get_environment(env_name)

jit_reset = jax.jit(eval_env.reset)
jit_step = jax.jit(eval_env.step)

In [ ]:
# initialize the state
rng = jax.random.PRNGKey(0)
state = jit_reset(rng)
rollout = [state.pipeline_state]

# grab a trajectory
n_steps = 500
render_every = 2

for i in range(n_steps):
  act_rng, rng = jax.random.split(rng)
  ctrl, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_step(state, ctrl)
  rollout.append(state.pipeline_state)

  if state.done:
    break

media.show_video(env.render(rollout[::render_every]), fps=1.0 / env.dt / render_every)